In [2]:
# Import statements
import pandas as pd, requests, json
import numpy as np
import math

In [3]:
# Load in the data
loans = pd.read_csv('./sample_ppp_loan_data.csv')
loans.head()

,LoanRange,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsRetained,DateApproved,Lender,CD,ObjectId,x,y
0,a $5-10 million,SCHULZE AND BURCH BISCUIT CO,1133 W 35th St,Chicago,IL,60609.0,311812.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,500.0,5/3/20 0:00,"Bank of America, National Association",IL - 01,1.0,-87.655059,41.829248
1,b $2-5 million,TRAILBOSS TX LLC,201 E 3rd Avenue,ANCHORAGE,AK,99501.0,561210.0,Limited Liability Company(LLC),Black or African America,Male Owned,Veteran,NaN,199.0,4/15/2020 20:00:00 AM,Alaska USA FCU,AK-00,13.0,-149.880549,61.219722
2,c $1-2 million,"ALASKA RENT-A-CAR, INC",4900 s aircraft driv,ANCHORAGE,AK,99502.0,532111.0,Employee Stock Ownership Plan(ESOP),Unanswered,Female Owned,Non-Veteran,NaN,18.0,4/14/20 0:00,First National Bank Alaska,AK-00,29.0,-149.968994,61.176305
3,"d $350,000-1 million",WESTERN WYOMING BEVERAGES INC,100 RELIANCE ROAD,ROCK SPRINGS,WY,82901.0,424810.0,Corporation,White,Male Owned,Non-Veteran,NaN,72.0,4/27/20 0:00,RSNB Bank,WY-00,661884.0,-109.215622,41.661937
4,"e $150,000-350,000",OLD SAN JUAN BAKERY INC.,2460 Main Street,SPRINGFIELD,MA,1107.0,311811.0,Corporation,Puerto Rican,Male Owned,Non-Veteran,NaN,17.0,7/6/20 0:00,Arrha CU,MA-01,289043.0,-72.604422,42.111811


In [3]:
# Transform panda database into python dictionary
def csvToJson(data, properties, lat="latitude", long="longitude"):
    geojson = {"type":"FeatureCollection", "features":[]}

    for row in data.itertuples():
        feature = {"type":"Feature",
                       "properties":{},
                       "geometry":{"type":"Point",
                                   "coordinates":[]}}

        feature["geometry"]["coordinates"] = [getattr(row, lat),getattr(row, long)]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            element = getattr(row, prop)
            if (isinstance(element, float) and math.isnan(element)):
                feature["properties"][prop] = 'Unanswered'
            else:
                feature["properties"][prop] = element

        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson["features"].append(feature)
        
    return geojson

# Split datatable into array of smaller dataframes by column value
def split_data(data, column_to_split, categories):
    databases = []
    for category in categories:
        file = data.loc[data[column_to_split] == category]
        databases.append(file)
    return databases

In [5]:
columns = ["LoanRange", "BusinessName", "BusinessType", "RaceEthnicity", "Gender", "Lender"]
amounts = ['a $5-10 million', 'b $2-5 million', 'c $1-2 million', 'd $350,000-1 million', 'e $150,000-350,000']

# Split data by loan type
datasets = split_data(loans, 'LoanRange', amounts)

# Output datasets into multiple files
for i, file in enumerate(datasets):
    file_string = 'loans_' + amounts[i][0] + '.json'
    with open(file_string, 'w') as fileout:
        json.dump(csvToJson(file, columns), fileout)

In [4]:
# Combine split datasets into single dictionary with different keys and output a single file
final_dataset = {'a': csvToJson(datasets[0], columns), 'b': csvToJson(datasets[1], columns), 'c': csvToJson(datasets[2], columns), 'd': csvToJson(datasets[3], columns), 'e': csvToJson(datasets[4], columns)}
with open('complete_loan_data.json', 'w') as fileout:
    json.dump(final_dataset, fileout)

In [ ]:
# Output datasets into multiple files
for i, file in enumerate(datasets):
    file_string = 'loans_' + amounts[i][0] + '.json'
    with open(file_string, 'w') as fileout:
        json.dump(csvToJson(file, columns), fileout)